In [1]:
from paddleocr import PaddleOCR
import numpy as np
from sklearn.cluster import KMeans

# OCR 모델 초기화 -> False로 한 이유는 속도 향상으로
ocr = PaddleOCR(
    use_doc_orientation_classify=False,  # 문서 기울기 파악해서 보정할건지
    use_doc_unwarping=False,             # 문서 구겨지거나 왜곡된거 펼건지
    use_textline_orientation=False,      # 글자 한 줄 한 줄 기울기 파악해서 보정할건지 
    lang="korean"
)

def extract_text(image_path, y_threshold=15, diff_threshold=600, balance_ratio=0.4):  # y_threshold=15, diff_threshold=600, balance_ratio=0.4 
    results = ocr.predict(input=image_path)
    
    if not results or not results[0]:
        return []

    result = results[0]
    boxes = result.get("rec_boxes", [])
    texts = result.get("rec_texts", [])
    
    if isinstance(boxes, np.ndarray):
        boxes = boxes.tolist()
        
    if not boxes:
        return []
    
    center_points = []
    for box in boxes:
        pts = np.array(box).reshape(-1, 2)
        cx = np.mean(pts[:, 0])
        cy = np.mean(pts[:, 1])
        center_points.append((cx, cy))

    X = np.array([[cx] for cx, _ in center_points])

    # K-Means를 통해 칼럼을 감지하는 로직
    is_column_layout = False
    labels = [0] * len(center_points)
    
    if len(center_points) > 4:
        kmeans = KMeans(n_clusters=2, random_state=0, n_init=10).fit(X)
        labels = kmeans.labels_
        centers = kmeans.cluster_centers_.flatten()
        diff = abs(centers[0] - centers[1])

        counts = [sum(labels == i) for i in range(2)]
        left_ratio = min(counts) / max(counts) if max(counts) > 0 else 0

  
        if diff > diff_threshold and left_ratio >= balance_ratio:
            is_column_layout = True

    def sort_and_group(items, y_threshold=15):
        items.sort(key=lambda x: (x[0], x[1]))
        grouped, current_line, last_y = [], [], None
        for cy, cx, text in items:
            if last_y is None or abs(cy - last_y) <= y_threshold:
                current_line.append((cx, text))
            else:
                current_line.sort(key=lambda x: x[0])
                grouped.append(" ".join([t for _, t in current_line]))
                current_line = [(cx, text)]
            last_y = cy
        if current_line:
            current_line.sort(key=lambda x: x[0])
            grouped.append(" ".join([t for _, t in current_line]))
        return grouped

    if is_column_layout:
        left_label = np.argmin(centers)
        left_items, right_items = [], []
        for (cx, cy), text, label in zip(center_points, texts, labels):
            if label == left_label:
                left_items.append((cy, cx, text))
            else:
                right_items.append((cy, cx, text))
        left_lines = sort_and_group(left_items, y_threshold)
        right_lines = sort_and_group(right_items, y_threshold)
        ordered_texts = left_lines + right_lines
    else:
        items = [(cy, cx, text) for (cx, cy), text in zip(center_points, texts)]
        ordered_texts = sort_and_group(items, y_threshold)

    return ordered_texts

# 테스트
image1 = "C:/Potenup/Drug-Detection-Chatbot/data/medicine_00806.jpeg"
#image2 = "C:/Potenup/Drug-Detection-Chatbot/data/medicine_00282.jpg"

print("==== 첫번째 이미지 ====")
for line in extract_text(image1):
    print(line)

# print("\n==== 두번째 이미지 ====")
# for line in extract_text(image2):
#     print(line)

c:\Potenup\Drug-Detection-Chatbot\.venv\lib\site-packages\paddle\utils\cpp_extension\extension_utils.py:718: UserWarning: No ccache found. Please be aware that recompiling all source files may be required. You can download and install ccache from: https://github.com/ccache/ccache/blob/master/doc/INSTALL.md
  warnings.warn(warning_message)
Creating model: ('PP-OCRv5_server_det', None)
Model files already exist. Using cached files. To redownload, please delete the directory manually: `C:\Users\user\.paddlex\official_models\PP-OCRv5_server_det`.
Creating model: ('korean_PP-OCRv5_mobile_rec', None)
Model files already exist. Using cached files. To redownload, please delete the directory manually: `C:\Users\user\.paddlex\official_models\korean_PP-OCRv5_mobile_rec`.


==== 첫번째 이미지 ====
[원료약품 및 분량] 이 약 1캡슐 중
유효성분:두타스테리드(USP)0.5mg
첨가제(동물유래성분):젤라틴(기원동물:소,사용
부위:가죽)
기타첨가제:농글리세린,부틸히드록시톨루엔,숙
신산젤라틴,폴리옥실40경화피마자유,프로필렌글
리콜모노라우레이트,폴록사머124,D-소르비톨액
[성상]
무색투명한 내용물이 들어있는 미황색의 투명한
타원형 연질캡슐
[효능·효과]
양성 전립선 비대증 증상의 개선,급성 성 요저류 위험성
감소,양성 전립선 비대증과 관련된 수술 필요성 감소,
성인 남성(만18~50세)의 남성형 탈모(androgenetic
alopecia)의 치료
[용법·용량]
이 약의 권장용량은 1일 1회 1캡슐(0.5mg)이다.
캡슐 내용물에 노출시 구강 인두점막의 자극을 초래
할 수 있으므로 이 약을 쉽거나 쪼개지 않고 통째로
삼켜 복용해야 한다. 이 약은 식사와 관계없이 복용할
수 있다.신장애 환자 또는 노인 환자에서 이 약의
용량을 조절할 필요는 없다. 간장애 환자에게 이 약을
투여한 자료가 없기 때문에 간장애 환자에서의 이
약의 권장용량은 확립되어 있지 않다.
[사용상의 주의사항]
1.경고1)여성에게 노출시 남자 태아에 미치는 위험성 이 약은
피부를 통 통해서 서 흡수된다.따라서 이 약의 흡수 가능성과 남자
태아에게 미치는 태자 기형의 위험 가능성 때문에 임신했거나
임신기 가능성이 있는 여성이 이 약을 취급해서는 안든 된다. 또,
여성은 이 약을 취급할 때마다 주의해야 하고 누출되는 캡슐
과의 접촉을 피해야 한다. 만약 캡슐이 새어 이 약과 접촉한
경우에는 접촉부위를 즉시 물과 비누로 세척해야 한다.
이하 첨부문서 참조
[저장방법] 밀폐용기,30°C이하 보관
[제조의뢰자](주)제뉴원사이언스
세종특별자치시 I|전의면 산단길 245
[제조자](주)유유제약
충북 제천시 바이오밸리 1로 94
